# LioNets Test Example
Testing LioNets Architecture on SMS Spam Collection dataset and Hate Speech dataset

In [ ]:
from keras import Sequential
from keras.engine.saving import model_from_json
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.layers import Input, Dense, Embedding, Conv1D, MaxPooling1D, Dropout, LSTM, RepeatVector, GlobalMaxPooling1D, \
    Concatenate, UpSampling2D, UpSampling1D, concatenate
from keras.models import Model
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from load_dataset import Load_Dataset
from IPython.display import Image
import numpy as np

## Setup process (predictor, encoder, decoder)
We load the datasets using a python script to do preprocessing

In [ ]:
#X,y,class_names = Load_Dataset.load_hate_speech() #Uncomment to test LioNets on Hate Speech Dataset
X,y,class_names = Load_Dataset.load_smsspam() #Uncomment to test LioNets on SMS Spam Collection

Then we split in train and test data. 

In [ ]:
X_train, X_test ,y_train ,y_test = train_test_split(X,y, random_state=70, stratify=y, test_size=0.33)
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

As vectorizing technique we will use the TF-IDF vectorizer with maximum amount of words/features 4000

In [ ]:
vec = TfidfVectorizer(max_features=4000)
vec.fit(X_train_copy)
X_train_copy = vec.transform(X_train_copy)
X_test_copy = vec.transform(X_test_copy)

Then we will create and train the classifier (the predictor)

In [ ]:
input_dim = len(vec.get_feature_names())
encoder_input = Input(shape=(input_dim,))
encoder_x = Dense(800, activation='tanh')(encoder_input)
encoder_x = Dropout(0.2)(encoder_x)
encoder_x = Dense(600, activation='tanh')(encoder_x)
encoder_x = Dense(400, activation='tanh')(encoder_x)
predictions = Dense(1, activation='sigmoid')(encoder_x)
predictor = Model(encoder_input,predictions)
predictor.compile(optimizer="adam",loss=["binary_crossentropy"],metrics=['accuracy'])
print(predictor.summary())
predictor.fit([X_train_copy], [y_train], validation_data=(X_test_copy,y_test), epochs=2, verbose=2)  # starts training
y_preds = predictor.predict(X_test_copy)

y_pred = [0 if a<0.5 else 1 for a in y_preds]
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

Let's plot our predictors architecture as well

In [ ]:
plot_model(predictor, 'Predictor.png',show_shapes=True)
Image(retina=True, filename='Predictor.png')

Now let's extract the encoder from the predictor. We will extract all the layers sequentially till the penultimate layer. Moreover, we set the weights of the encoder untrainable in order to preserve the acquired knowledge, which the neural network inferred through its training process.

In [ ]:
encoder = Sequential()
for i in range(0,len(predictor.layers)-1):
    print(predictor.layers[i])
    encoder.add(predictor.layers[i])
encoder.summary()
encoder.trainable = False

Lastly, we form and we train the decoder model through an autoencoder. We will use the trained encoder as the first half of the autoencoder.

In [ ]:
input_dim = len(vec.get_feature_names())
encoder_input = Input(shape=(input_dim,))
encoded_x = encoder(encoder_input)
decoder_x = Dense(600, activation='tanh')(encoded_x)
decoder_x = Dense(800, activation='tanh')(decoder_x)
decoder_x = Dropout(0.5)(decoder_x)
decoded = Dense(input_dim, activation='softmax')(decoder_x)
autoencoder = Model(encoder_input,decoded)
autoencoder.compile(optimizer='adam',loss=["categorical_crossentropy"],metrics=['accuracy'])#Did try MAE, MSY as well

print(autoencoder.summary())
autoencoder.fit([X_train_copy], [X_train_copy], validation_data=(X_test_copy,X_test_copy), epochs=150, verbose=2)  #Start training

We plot the autoencoder

In [ ]:
plot_model(autoencoder, 'Autoencoder.png',show_shapes=True)
Image(retina=True, filename='Autoencoder.png')

And we extract the decoder

In [ ]:
decoder = Sequential()
for i in range(2,len(autoencoder.layers)):
    decoder.add(autoencoder.layers[i])
decoder(optimizer='adam',loss=["categorical_crossentropy"],metrics=['accuracy'])#Mae, mse try

We plot the decoder

In [ ]:
plot_model(autoencoder, 'Decoder.png',show_shapes=True)
Image(retina=True, filename='Decoder.png')

## LioNets Experiments
Having everything setted up, we are now ready to try our methodology.

In [ ]:
from LioNets import LioNet
from lime.lime_text import LimeTextExplainer
from collections import OrderedDict
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

Set an instance id to get predictions and produce an explanation

In [ ]:
#idx = 5# For SMS Spam
idx =10# For Hate Speech

We initialize LioNets, giving the predictor, decoder, encoder, as well as the feature names, as arguments, we are ready to extract an explanation.

In [ ]:
lionet = LioNet(model=predictor, autoencoder=None, decoder=decoder, encoder=encoder, feature_names=vec.get_feature_names())
print(X_train[idx])
lionet.explain_instance(X_train_copy[idx])
lionet.print_neighbourhood_labels_distribution()

We want to compare LioNets' explanation with LIME's explanation. So we set up LIME in order to produce One more explanation

In [ ]:
def li_predict2(text):
    texts = vec.transform(text)
    a = predictor.predict(texts)
    b = 1 - a 
    return np.column_stack((b,a))

text = X_train[idx]
split_expression = lambda s: re.split(r'\W+', s)
explainer = LimeTextExplainer(class_names=class_names, split_expression=split_expression)
explanation = explainer.explain_instance(text_instance=text, classifier_fn=li_predict2)
weights = OrderedDict(explanation.as_list())
lime_w = pd.DataFrame({'Features': list(weights.keys()), "Features' Weights" : list(weights.values())})
plt.figure(num=None, figsize=(6, 6), dpi=200, facecolor='w', edgecolor='k')
lime_w = lime_w.sort_values(by="Features' Weights", ascending=False)
sns.barplot(x="Features' Weights", y="Features", data=lime_w)
plt.xticks(rotation=90)
print('Instance:',X_train[idx])
plt.show()

We vizualize the LIME's explanation with LIME's tools

In [ ]:
explanation.save_to_file('/tmp/oi.html')

In [ ]:
explanation.show_in_notebook(text=True)

## Neighbours Distances

We are going to compute the distances between neighbours on the original space and in the reduced space. Firstly, we will apply this to an instance of train set

First of all we use the encoder to encode our data to the encoded space. Thus, we are reducing their dimensions.

In [ ]:
encoded_X_train = encoder.predict(X_train_copy)
encoded_X_test = encoder.predict(X_test_copy)

In [ ]:
decoded_X_train = decoder.predict(encoded_X_train)

Set an instance id to get predictions and produce an explanation

In [ ]:
#ida = 5# For SMS Spam
ida =10# For Hate Speech

Take the instance with 

In [ ]:
encoded_instance = encoded_X_train[ida] #Also we take the encoded instance
decoded_instance = decoder.predict(encoded_X_train)[ida] #And the decoded instance

We print the decoded instance to see the performance of the autoencoder

In [ ]:
decoded_instance_cleaned = []
for i in decoded_instance:
    if i < 0.01: decoded_instance_cleaned.append(0)
    else: decoded_instance_cleaned.append(i)
print(" ".join(vec.inverse_transform([decoded_instance_cleaned])[0]))

We will use the below metrics

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

Which is the highest elemend and its index in the vector?

In [ ]:
max = list(X_train_copy[ida].copy().A[0]).index(X_train_copy[ida].copy().A[0].max())
print(list(X_train_copy[ida].copy().A[0]).index(X_train_copy[ida].copy().A[0].max()))#Index of highest value
initial_instance = X_train_copy[ida].copy().A[0]
generated_instance = X_train_copy[ida].copy().A[0]
print("Value on initial instance:",generated_instance[max]) #Highest value
generated_instance[max] = 0
print("Value on generated instance:",generated_instance[max]) #New value

Let's compute the distances between the initial and the generated instance. They only differ in one feature!

In [ ]:
print("Euclidean Distance:",euclidean_distances([initial_instance],[generated_instance])[0][0])

Now let's create a neighbourhood and find the distances between them in the original space (That's what LIME does but only by putting one zero at a time). This will create only n neighbours (where n the number of words of the sentence)

In [ ]:
cosSim = 0
cosDis = 0
eucDis = 0
manDis = 0
neighbours = []
count = 0
for i in range(0,len(initial_instance)):
    if(initial_instance[i]!=0):
        gen = initial_instance.copy()
        gen[i]=0
        eucDis = eucDis + euclidean_distances([initial_instance],[gen])[0][0]
        count = count + 1
        neighbours.append(gen)
print("Euclidean Distance:",eucDis/count)
print(count)

Let's encode them and reduce their dimensions to 400! And Let's compute the new distances between the initial and the generated instance in the reduced space.

In [ ]:
initial_encoded_instance = encoded_X_train[ida]
generated_encoded_instance = encoder.predict(np.array([generated_instance]))[0]
print("Euclidean Distance:",euclidean_distances([initial_encoded_instance],[generated_encoded_instance])[0][0])

Let's compute the distances in the reduced space

In [ ]:
cosSim = 0
cosDis = 0
eucDis = 0
manDis = 0
count = 0
for i in neighbours:
    generated_encoded_instance = encoder.predict(np.array([i]))[0]
    eucDis = eucDis + euclidean_distances([initial_encoded_instance],[generated_encoded_instance])[0][0]
    count = count + 1
print("Euclidean Distance:",eucDis/count)
print(count)

Which is the highest element and its index in the reduced vector?

In [ ]:
max = list(encoded_X_train[ida].copy()).index(encoded_X_train[ida].copy().max())
print(list(encoded_X_train[ida].copy()).index(encoded_X_train[ida].copy().max()))
initial_encoded_instance = encoded_X_train[ida].copy()
generated_encoded_instance = encoded_X_train[ida].copy()
print(generated_encoded_instance[max])
generated_encoded_instance[max]=0
print(generated_encoded_instance[max])

In [ ]:
print("Euclidean Distance:",euclidean_distances([initial_encoded_instance],[generated_encoded_instance])[0][0])

Now we will create neighbours (more than before in the reduced space) and we will compute their distances from the initial encoded instance

In [ ]:
cosSim = 0
cosDis = 0
eucDis = 0
manDis = 0
neighbours = []
count = 0
for i in range(0,len(initial_encoded_instance)):
    if(initial_encoded_instance[i]!=0):
        gen = initial_encoded_instance.copy()
        gen[i]=0
        eucDis = eucDis + euclidean_distances([initial_encoded_instance],[gen])[0][0]
        count = count + 1
        neighbours.append(gen)
print("Euclidean Distance:",eucDis/count)
print(count)

In [ ]:
initial_instance = X_train_copy[ida].A[0]
generated_instance = decoder.predict(np.array([generated_encoded_instance]))[0]
print("Euclidean Distance:",euclidean_distances([initial_instance],[generated_instance])[0][0])

And we will find the differences on the original space after the transformation (through the decoder)

In [ ]:
cosSim = 0
cosDis = 0
eucDis = 0
manDis = 0
count = 0
for i in neighbours:
    generated_decoded_instance = decoder.predict(np.array([i]))[0]
    eucDis = eucDis + euclidean_distances([initial_instance],[generated_decoded_instance])[0][0]
    count = count + 1
print("Euclidean Distance:",eucDis/count)
print(count)